In [1]:
from __future__ import division, print_function
import os
import numpy as np
from flask import Flask, request, render_template
from werkzeug.utils import secure_filename
from python_speech_features import mfcc
import scipy.io.wavfile as wav
import pickle
import operator

In [2]:
# Define a flask app
app = Flask(__name__)

dataset = []
def loadDataset(filename):
    with open("my.dat" , 'rb') as f:
        while True:
            try:
                dataset.append(pickle.load(f))
            except EOFError:
                f.close()
                break

loadDataset("my.dat")


In [3]:
def distance(instance1 , instance2 , k ):
    distance =0 
    mm1 = instance1[0] 
    cm1 = instance1[1]
    mm2 = instance2[0]
    cm2 = instance2[1]
    
    #Method to calculate distance between two instances.
    distance = np.trace(np.dot(np.linalg.inv(cm2), cm1)) 
    distance+=(np.dot(np.dot((mm2-mm1).transpose() , np.linalg.inv(cm2)) , mm2-mm1 )) 
    distance+= np.log(np.linalg.det(cm2)) - np.log(np.linalg.det(cm1))
    distance-= k
    print("distane is",distance)
    return distance

In [4]:
def getNeighbors(trainingSet , instance , k):
    distances =[]
    for x in range (len(trainingSet)):
        dist = distance(trainingSet[x], instance, k )+ distance(instance, trainingSet[x], k)
        distances.append((trainingSet[x][2], dist))
    distances.sort(key=operator.itemgetter(1))
    neighbors = []
    for x in range(k):
        neighbors.append(distances[x][0])
    print("neighbors is ",neighbors)
    return neighbors   

In [5]:
def nearestClass(neighbors):
    classVote ={}
    for x in range(len(neighbors)):
        response = neighbors[x]
        if response in classVote:
            classVote[response]+=1 
        else:
            classVote[response]=1 
    sorter = sorted(classVote.items(), key = operator.itemgetter(1), reverse=True)
    print("sorter is ",sorter)
    return sorter[0][0]


In [6]:
print('Model loaded. Check http://127.0.0.1:5000/')


@app.route('/', methods=['GET'])
def index():
    # Main page
    return render_template('music.html')


Model loaded. Check http://127.0.0.1:5000/


In [7]:
@app.route('/predict', methods=['GET', 'POST'])
def upload():
    if request.method == 'POST':
        # Get the file from post request
        f = request.files['image']
        
        # Save the file to ./uploads
        basepath = "E:\TSB projects\Flask"
        file_path = os.path.join(basepath, 'uploads', secure_filename(f.filename))
        f.save(file_path)
        print(file_path)
        i=1
        
        results = {1: 'blues', 2: 'classical', 3: 'country', 4: 'disco', 5: 'hiphop', 
                   6: 'jazz', 7: 'metal', 8: 'pop', 9: 'reggae', 10: 'rock'}
    
        (rate,sig)=wav.read(file_path)
        print(rate,sig)
        mfcc_feat=mfcc(sig,rate,winlen=0.020,appendEnergy=False)
        covariance = np.cov(np.matrix.transpose(mfcc_feat))
        mean_matrix = mfcc_feat.mean(0)
        feature=(mean_matrix,covariance,0)
        pred=nearestClass(getNeighbors(dataset ,feature , 8))
        
        print("predicted genre = ",pred,"class = ",results[pred])
        return "This song is classified as a "+str(results[pred])


In [ ]:
if __name__ == '__main__':
    app.run(threaded = False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [31/May/2021 21:57:57] "GET / HTTP/1.1" 200 -


E:\TSB projects\Flask\uploads\classical.00004.wav
22050 [ -353  -656  -795 ... -3446 -3623 -3008]
distane is 37.15191526270923
distane is 26.341506474825643
distane is 51.28330212259906
distane is 24.810705008338587
distane is 41.3730091969094
distane is 30.097487964292263
distane is 45.210471587732876
distane is 27.930380339300733
distane is 23.150282494165026
distane is 14.838007596823722
distane is 37.47702754840294
distane is 24.38871014703207
distane is 45.009803572013695
distane is 21.629090737788687
distane is 40.08565451149091
distane is 26.131131177425303
distane is 48.811803817367675
distane is 43.291126325393215
distane is 37.05910991704111
distane is 23.590012013155242
distane is 41.60055753205741
distane is 20.37047360585904
distane is 51.23422333639354
distane is 28.858786650853773
distane is 37.28332280121289
distane is 31.329932281858383
distane is 46.82616346584149
distane is 69.88586313192876
distane is 65.75915719979466
distane is 32.89725914580091
distane is 37.9089

distane is 33.51767418492309
distane is 49.7538873038135
distane is 37.8285746918204
distane is 39.19091531853582
distane is 21.698379067584106
distane is 43.77003422541794
distane is 34.91084747501453
distane is 59.03823592052933
distane is 42.591669301449265
distane is 49.570619684945775
distane is 42.1585614059647
distane is 37.150202808790404
distane is 21.335850659720336
distane is 59.99913387658101
distane is 32.49079129863074
distane is 31.2794177362304
distane is 16.58533719999912
distane is 42.343004102513945
distane is 24.58941351369308
distane is 50.02669213011517
distane is 35.0616035432969
distane is 42.50331643205188
distane is 16.49026670034235
distane is 58.81586229691942
distane is 25.681576363664057
distane is 41.186564020432115
distane is 20.964211296632868
distane is 42.04283443280241
distane is 28.32235675733788
distane is 83.8299835686884
distane is 46.40647431926833
distane is 41.598292986625374
distane is 30.483597073267596
distane is 39.386848896065544
distane 

distane is 104.24520735583553
distane is 56.514908258240396
distane is 82.16787619810569
distane is 55.56382586379243
distane is 69.24712849717778
distane is 42.91835823026122
distane is 83.00604058169995
distane is 63.000223513822974
distane is 130.6720043127477
distane is 121.95428629342817
distane is 43.18877679786052
distane is 39.56877714939233
distane is 120.93768565039426
distane is 59.19530348777181
distane is 115.04135737839023
distane is 64.24135366815995
distane is 88.54390285041333
distane is 71.03331767926406
distane is 143.61782833541713
distane is 79.24274311359643
distane is 111.08290069883452
distane is 59.047646642913875
distane is 133.90790451032433
distane is 77.64518368674342
distane is 115.46476568943851
distane is 57.64740586193406
distane is 120.85750300646725
distane is 58.98382120426055
distane is 143.00811216870144
distane is 56.715080226273685
distane is 124.7483770177698
distane is 59.318491596718246
distane is 140.48077219711575
distane is 42.4632632721742

127.0.0.1 - - [31/May/2021 21:58:14] "POST /predict HTTP/1.1" 200 -


 68.35560578862513
distane is 32.06475757067903
distane is 47.30883053223413
distane is 43.585584663983326
distane is 81.35794756942539
distane is 32.75544798940504
distane is 52.39760221236956
distane is 29.339216543491304
distane is 44.001996727053495
distane is 25.611209515024512
distane is 37.27937884516313
distane is 42.83070723807584
distane is 34.114565726190115
distane is 30.488117523007823
distane is 44.167810898653414
neighbors is  [2, 2, 2, 2, 2, 2, 2, 2]
sorter is  [(2, 8)]
predicted genre =  2 class =  classical
